In [4]:
import math
import pickle
import random
import sys
from multiprocessing import Pool

import networkx as nx
import numpy as np
import pandas as pd
from scipy.spatial import KDTree as kd
from tqdm import trange, tqdm
import matplotlib.pyplot as plt
import city_tests
import graph_generator

import osmnx as ox


In [5]:


def get_graph(city_id: str = 'R2555133') -> nx.Graph:
    gdf = ox.geocode_to_gdf(city_id, by_osmid=True)
    polygon_boundary = gdf.unary_union
    graph = ox.graph_from_polygon(polygon_boundary,
                                  network_type='drive',
                                  simplify=True)
    G = nx.Graph(graph)
    H = nx.Graph()
    # Добавляем рёбра в новый граф, копируя только веса
    for u, d in G.nodes(data=True):
        H.add_node(u, x=d['x'], y=d['y'])
    for u, v, d in G.edges(data=True):
        H.add_edge(u, v, length=d['length'])
    del city_id, gdf, polygon_boundary, graph, G
    return H


def calculate(data):
    cities = data[0]
    points_number = data[1]
    NUMBER = data[2]
    THREADS = data[3]

    for name, id in cities:
        G = get_graph(id)

        points = [graph_generator.get_node_for_initial_graph_v2(G) for _ in
                  range(points_number)]

        Q = G.copy()
        for u in Q.nodes:
            if u in Q[u]:
                Q.remove_edge(u, u)
        data = {}
        for (u, d) in Q.degree:
            if d not in data:
                data[d] = 0
            data[d] += 1
        dd = {
            'degree': [],
            'probability': []
        }
        total = sum([d for d in data.values()])

        for d in data:
            dd['degree'].append(d)
            dd['probability'].append(data[d] / total)
        df = pd.DataFrame.from_dict(dd)
        print(df)
        return df

In [6]:


if __name__ == '__main__':
    total = 1
    points_number = 500
    if len(sys.argv) == 2:
        total = int(sys.argv[1])

    # print('THREADS:', total)
    # print('POINTS:', points_number)

    cities = {
        # 'ASHA': 'R13470549',
        # 'KRG': 'R4676636',
        # 'EKB': 'R6564910',
        # 'BARCELONA': 'R347950',
        'PARIS': 'R71525',
        # 'Prague': 'R435514',
        # 'MSK': 'R2555133',
        # 'SBP': 'R337422',
        # 'SINGAPORE': 'R17140517',
        # 'BERLIN': 'R62422',
        # 'ROME': 'R41485',
        # 'LA': 'R207359',
        # 'DUBAI': 'R4479752',
        # 'RIO': 'R2697338',
        # 'DELHI': 'R1942586',
        # 'KAIR': 'R5466227'
    }
    total_len = len(cities)
    l = list(cities.items())
    data = [[l[i: total_len: total], points_number, i + 1, total] for i in range(total)]

    df = calculate(data[0])

   degree  probability
0       4     0.230009
1       3     0.655249
2       2     0.040139
3       5     0.016707
4       1     0.056530
5       6     0.001261
6       7     0.000105


In [7]:
df

,degree,probability
0,4,0.230009
1,3,0.655249
2,2,0.040139
3,5,0.016707
4,1,0.056530
5,6,0.001261
6,7,0.000105


In [8]:
momentums = {}
for i in range(1, 10):
    momentums[i] = 0
    for j in range(len(df)):
        momentums[i] += (df['degree'][j] ** i) * df['probability'][j]


In [9]:
for m in momentums:
    print(m, ':', momentums[m])

1 : 3.114426815172849
2 : 10.262687821792586
3 : 35.18671850373017
4 : 124.9845539560786
5 : 459.87601134811393
6 : 1754.656929704739
7 : 6951.433434905958
8 : 28633.035410318378
9 : 122791.35862141431


In [10]:
degree_destrib = {
    0: 0,
    3: 0.816342,
    2: 0.044013,
    1: 0.071698,
    4: 0.061802,
    5: 0.005618,
    7: 0.000017
}

q = list(degree_destrib.items())
q.sort(key=lambda x: x[0])
degree_destrib = dict(q)
W = list(degree_destrib.values())

W = np.array(W)
W/=np.sum(W)
print(W)


[0.00000000e+00 7.17345846e-02 4.40354581e-02 8.16758547e-01
 6.18335351e-02 5.62086664e-03 1.70086744e-05]


In [11]:
S = nx.expected_degree_graph(W)

In [12]:
# print(len(S.nodes))

In [13]:
print(sum(W))

1.0


In [14]:
print(W)

[0.00000000e+00 7.17345846e-02 4.40354581e-02 8.16758547e-01
 6.18335351e-02 5.62086664e-03 1.70086744e-05]


In [15]:
# print(c)

NameError: name 'c' is not defined

In [ ]:
N = 100
degrees = np.random.choice(range(0, len(W)), size=N,p=W)
print(degrees)